# Selenium 만으로 유튜브 랭킹 수집하기

In [24]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [26]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [27]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [28]:
# pop-up 창이 있으면, pop-up창을 닫고 메인 윈도우로 되돌아옴 
window_len = len(driver.window_handles)
if window_len == 2:
  driver.switch_to_window(driver.window_handles[1])
  driver.close()

# 화면의 끝까지 PageDown으로 내려보냄 
driver.switch_to_window(driver.window_handles[0])
body = driver.find_element_by_css_selector('body')
for i in range(30):
  body.send_keys(Keys.PAGE_DOWN)
  time.sleep(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  


In [29]:
divs = driver.find_elements_by_css_selector('.aos-init')
len(divs)

100

In [30]:
div = divs[0]
category = div.find_element_by_css_selector('.category').text.strip(' \n[]')
category

'음악/댄스/가수'

In [31]:
name = div.find_element_by_css_selector('.subject a').text
name

'BLACKPINK'

In [32]:
subscriber = div.find_element_by_css_selector('.subscriber_cnt').text
view = div.find_element_by_css_selector('.view_cnt').text
video = div.find_element_by_css_selector('.video_cnt').text
subscriber, view, video

('6400만', '190억0381만', '371개')

In [33]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [34]:
# 작업 정도를 표기

from tqdm.notebook import tqdm

In [36]:
results = []
for page in tqdm(range(1,11)):
    #print(page)
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)

    window_len = len(driver.window_handles)
    if window_len == 2:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    trs = driver.find_elements_by_css_selector('.aos-init')

    for tr in trs:
        category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
        name = tr.find_element_by_css_selector('.subject a').text.strip()
        subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
        view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
        video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [37]:
df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,애완/반려동물,크림히어로즈,"3,800,000","1,374,770,000",940
1,BJ/인물/연예인,워크맨-Workman,"3,780,000","679,370,000",141
2,BJ/인물/연예인,waveya 2011,"3,730,000","1,346,280,000",702
3,키즈/어린이,Lime Tube[라임튜브],"3,730,000","2,595,420,000","1,723"
4,음악/댄스/가수,WINNER,"3,730,000","1,171,260,000",428


In [38]:
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
895,미분류,밉지않은 관종언니,"510,000","67,950,000",170
896,게임,미소,"510,000","210,780,000","3,084"
897,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
898,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
899,게임,임선비,"510,000","234,290,000","1,134"


In [39]:
driver.close()